In [1]:
import os
import pandas as pd
from datetime import datetime as dt
import numpy as np

In [6]:
print(os.getcwd())

/Users/ZiweiMengyang/Desktop/Python & Machine Learning/Tiger/Capstone/Codes


# Step 1. Load csv into pandas DataFrame (play, down, search)

In [2]:
play = pd.read_csv('../Data/play_sample.csv', header=0)

In [5]:
play['play_date'] = pd.to_datetime(play['date'])
play.drop(columns=['date'], inplace=True)

In [6]:
play.head()

,device,song_id,song_type,play_time,song_length,uid,churn,play_date
0,ar,1373736.0,0.0,211.0,231.0,154708890,1,2017-03-30
1,ar,4811014.0,0.0,18.0,205.0,154708890,1,2017-03-30
2,ar,1131748.0,0.0,55.0,474.0,154708890,1,2017-03-30
3,ar,6364417.0,0.0,72.0,226.0,154708890,1,2017-03-30
4,ar,20865808.0,0.0,140.0,252.0,154708890,1,2017-04-08


In [44]:
# Check number of uid 
print(len(play['uid'].unique()))

21622


### Define end_date  
*1. For down and search data, first remove log > pd.to_date('2017-04-29')*

*2. Be careful to modify search timestamp to 00:00:00*

*3. Use datetime dtype because it is much more powerful than data dtype*

In [7]:
end_date = play['play_date'].max() 
end_date

Timestamp('2017-04-29 00:00:00')

In [8]:
search = pd.read_csv('../Data/search_sample.csv', header=0)

In [14]:
search['search_date'] = search['search_date_str'].apply(lambda x: dt.strptime(x[:10], '%Y-%m-%d'))
search.head()

,uid,search_date_str,search_query,search_date
0,168529773,2017-03-30 00:01:15,%e9%83%ad%e5%be%b7%e7%ba%b2%e4%ba%8e%e8%b0%a6%...,2017-03-30
1,168529776,2017-03-30 00:02:13,%E6%9E%97%E9%80%B8%E8%B6%85,2017-03-30
2,168529955,2017-03-30 00:03:18,super+junior,2017-03-30
3,168530160,2017-03-30 00:03:38,%e5%88%9a%e5%a5%bd%e9%81%87%e8%a7%81%e4%bd%a0,2017-03-30
4,168530474,2017-03-30 00:04:44,%E5%86%85%E8%92%99%E5%8F%A4%E9%BB%91%E6%80%95,2017-03-30


In [15]:
search = search[search['search_date'] <= end_date].drop(columns=['search_date_str', 'search_query'])
search['search_date'].max()

Timestamp('2017-04-29 00:00:00')

In [16]:
down = pd.read_csv('../Data/download_sample.csv', header=0)
down.dtypes

uid               int64
song_id_down    float64
date_down        object
dtype: object

In [17]:
down['down_date'] = pd.to_datetime(down['date_down'])
down = down[down['down_date'] <= end_date].drop(columns=['date_down'])
down['down_date'].max()

Timestamp('2017-04-29 00:00:00')

# Step 2. create Frequency and Avg. Play Time features for play data

In [30]:
def compute_play_freq(df, event, event_date, end_date, window):
    df_win = df[(end_date - df[event_date]).apply(lambda x: x.days <= window)]
    
    df_freq = df_win.groupby('uid')[event] \
                    .agg(['count']) \
                    .rename(columns={'count':'freq_last_' + str(window) + '_days_play'})
                                        
     
    df_daily_play_time = df_win.groupby(['uid'])[event].agg(['sum']) 
    
    df_daily_play_time['avg_daily_play_time_last_'+ str(window) + '_days'] = df_daily_play_time['sum']/window
    df_daily_play_time.drop(columns=['sum'], inplace=True)
    
    df_feature = df_freq.join(df_daily_play_time, how='outer')
    
    return df_feature

In [39]:
def compute_recency(df, event, event_date, end_date):
    
    last = df.groupby('uid')[event_date].agg(['max'])
    last['last_' + event] = (end_date - last['max']).apply(lambda x: x.days)
 
    last.drop(columns=['max'], inplace=True)
    return last

In [40]:
last_play = compute_recency(play, 'play', 'play_date', end_date)

In [41]:
last_play.head()

,last_play
uid,
103928,10
748140,1
1404537,2
1809207,2
3962203,3


In [31]:
play_3 = compute_play_freq(play, 'play_time', 'play_date', end_date, 3)

In [32]:
play_3.head(5)

,freq_last_3_days_play,avg_daily_play_time_last_3_days
uid,,
748140,14,576.666667
1404537,14,952.666667
1809207,1,78.666667
3962203,1,50.333333
4586045,6,1.666667


In [33]:
play_7 = compute_play_freq(play, 'play_time', 'play_date', end_date, 7)
play_14 = compute_play_freq(play, 'play_time', 'play_date', end_date, 14)
play_30 = compute_play_freq(play, 'play_time', 'play_date', end_date, 30)

###  - ***Merge play data together with down and search later in Step 5. ***

###  - ***Hold off Filling NaN until Step 5 to avoid repetitive labor ! ! ***

# Step 3. Create Frequency features for <u>*down*</u> and <u>*search*</u> data respectively

In [45]:
# function for down and search data 
def compute_freq(df, event, event_date, end_date, window):
    df_win = df[(end_date - df[event_date]).apply(lambda x: x.days <= window)]
    
    df_feature = df_win.groupby('uid')[event_date] \
                       .agg(['count']) \
                       .rename(columns={'count':'freq_last_' + str(window) + '_days_' + event})
                                       
    
    return df_feature

In [51]:
down_freq = compute_freq(down, 'download', 'down_date', end_date, 3)

In [47]:
down_freq.head()

,freq_last_3_days_download
uid,
5547484,2
7755737,1
21458356,18
22565317,2
22971804,1


In [52]:
for t in [7,14,30]:
    down_t = compute_freq(down, 'download', 'down_date', end_date, t)
    down_freq = down_freq.join(down_t, how='right') 


In [53]:
down_freq.head()

,freq_last_3_days_download,freq_last_7_days_download,freq_last_14_days_download,freq_last_30_days_download
uid,,,,
748140,NaN,NaN,NaN,13
1809207,NaN,NaN,1.0,3
3962203,NaN,2.0,2.0,8
4192045,NaN,NaN,NaN,4
4586045,NaN,NaN,3.0,15


In [55]:
last_down = compute_recency(down, 'download', 'down_date', end_date)
last_down.head()
# Validate last againt freq_last_x_days - helps detect datetime calculation error

,last_download
uid,
748140,22
1809207,14
3962203,6
4192045,24
4586045,11


In [57]:
search_freq = compute_freq(search, 'search', 'search_date', end_date, 3)
for t in [7,14,30]:
    search_t = compute_freq(search, 'search', 'search_date', end_date, t)
    search_freq = pd.merge(search_t, search_freq, how='left', left_index=True, right_index=True)

In [59]:
search_freq.head()

,freq_last_30_days_search,freq_last_14_days_search,freq_last_7_days_search,freq_last_3_days_search
uid,,,,
103928,35,16.0,NaN,NaN
748140,46,NaN,NaN,NaN
1404537,3,NaN,NaN,NaN
1809207,31,3.0,NaN,NaN
3962203,56,45.0,7.0,1.0


In [60]:
last_search = compute_recency(search, 'search', 'search_date', end_date)
last_search.head()
# Oppotunity to validate across different variables

,last_search
uid,
103928,10
748140,22
1404537,29
1809207,14
3962203,3


# Step 4. Merge play_feature, search_feature, down_feature together

In [61]:
play_check_1 = play[['uid', 'device', 'churn']].drop_duplicates()
play_check_1.shape

(21627, 3)

In [62]:
play_check_2 = play[['uid', 'churn']].drop_duplicates()
play_check_2.shape

(21622, 2)

* **Finding: 5 out of 22K users have multiple devices/switch devices during the 30 day obs window
  --> I won't create a new variable 'both' to capture this case because it is too rare. Instead, I just assign the first devices to these 5 users. **

In [74]:
play_feature = play[['uid', 'churn', 'device']].drop_duplicates().groupby(['uid']).first()

In [75]:
play_feature.head()

,churn,device
uid,,
103928,0,ar
748140,1,ar
1404537,0,ar
1809207,0,ar
3962203,1,ar


In [76]:
play_feature.shape

(21622, 2)

* **Create Dummy Variable for device** 

In [78]:
play_feature['device_ip'] = pd.get_dummies(play_feature[['device']], drop_first=True)
play_feature.drop(columns=['device'], inplace=True)
play_feature.head()

,churn,device_ip
uid,,
103928,0,0
748140,1,0
1404537,0,0
1809207,0,0
3962203,1,0


In [80]:
print('entire sample churn split is: \n')
print(play_feature['churn'].value_counts())

print('\n \n entire sample device split is: \n')
print(play_feature['device_ip'].value_counts())

entire sample churn split is: 

1    11554
0    10068
Name: churn, dtype: int64

 
 entire sample device split is: 

0    18794
1     2828
Name: device_ip, dtype: int64


In [81]:
play_feature = (play_feature.join(play_3, how='left')
                            .join(play_7, how='left')
                            .join(play_14, how='left')
                            .join(play_30, how='left')
                            .fillna(0)
               )

play_feature.head()

,churn,device_ip,freq_last_3_days_play,avg_daily_play_time_last_3_days,freq_last_7_days_play,avg_daily_play_time_last_7_days,freq_last_14_days_play,avg_daily_play_time_last_14_days,freq_last_30_days_play,avg_daily_play_time_last_30_days
uid,,,,,,,,,,
103928,0,0,0.0,0.000000,0.0,0.000000,6.0,32.071429,8,22.833333
748140,1,0,14.0,576.666667,14.0,247.142857,16.0,153.142857,332,1949.700000
1404537,0,0,14.0,952.666667,14.0,408.285714,25.0,346.428571,27,176.633333
1809207,0,0,1.0,78.666667,10.0,248.571429,51.0,486.785714,454,2185.600000
3962203,1,0,1.0,50.333333,32.0,282.857143,92.0,495.714286,128,313.933333


In [91]:
# Check data integrity: churn vs. active split still holds
play_feature['churn'].value_counts()

1    11554
0    10068
Name: churn, dtype: int64

In [82]:
# BE CAREFULE OF ORDER OF MERGE AND FILLNA ! ! 
# BECAUSE FREQ AND RECENCY IMPUTE MISSING VALUE DIFFERENTLY ! !

play_feature = play_feature.join(last_play, how='left').fillna(90)
play_feature.head()

,churn,device_ip,freq_last_3_days_play,avg_daily_play_time_last_3_days,freq_last_7_days_play,avg_daily_play_time_last_7_days,freq_last_14_days_play,avg_daily_play_time_last_14_days,freq_last_30_days_play,avg_daily_play_time_last_30_days,last_play
uid,,,,,,,,,,,
103928,0,0,0.0,0.000000,0.0,0.000000,6.0,32.071429,8,22.833333,10
748140,1,0,14.0,576.666667,14.0,247.142857,16.0,153.142857,332,1949.700000,1
1404537,0,0,14.0,952.666667,14.0,408.285714,25.0,346.428571,27,176.633333,2
1809207,0,0,1.0,78.666667,10.0,248.571429,51.0,486.785714,454,2185.600000,2
3962203,1,0,1.0,50.333333,32.0,282.857143,92.0,495.714286,128,313.933333,3


In [98]:
df_all = (play_feature.join(search_freq, how='left') 
                      .join(down_freq, how='left') 
                      .fillna(0))

print('Checking Churn Split')
df_all['churn'].value_counts()

Checking Churn Split


1    11554
0    10068
Name: churn, dtype: int64

In [99]:
df_all = pd.merge(df_all, last_search, left_index=True, right_index=True, how='left')
df_all = pd.merge(df_all, last_down, left_index=True, right_index=True, how='left')

df_all.fillna(90, inplace=True)

df_all.sample(20)

,churn,device_ip,freq_last_3_days_play,avg_daily_play_time_last_3_days,freq_last_7_days_play,avg_daily_play_time_last_7_days,freq_last_14_days_play,avg_daily_play_time_last_14_days,freq_last_30_days_play,avg_daily_play_time_last_30_days,...,freq_last_30_days_search,freq_last_14_days_search,freq_last_7_days_search,freq_last_3_days_search,freq_last_3_days_download,freq_last_7_days_download,freq_last_14_days_download,freq_last_30_days_download,last_search,last_download
uid,,,,,,,,,,,,,,,,,,,,,
168633447,1,0,0.0,0.000000,0.0,0.000000,0.0,0.000000,1,8.333333,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,90.0
168612811,1,0,4.0,244.333333,4.0,104.714286,4.0,52.357143,124,182.033333,...,7.0,0.0,0.0,0.0,2.0,2.0,2.0,5.0,23.0,0.0
168815586,1,0,0.0,0.000000,0.0,0.000000,0.0,0.000000,322,607.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,90.0,30.0
168333175,1,0,0.0,0.000000,0.0,0.000000,0.0,0.000000,708,1034.866667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,90.0,25.0
167936805,0,0,34.0,1760.333333,46.0,3178.428571,83.0,4313.285714,196,3109.833333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,90.0
168627360,1,0,0.0,0.000000,0.0,0.000000,0.0,0.000000,14,67.800000,...,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,90.0
168959102,1,0,0.0,0.000000,0.0,0.000000,0.0,0.000000,4,21.033333,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,90.0
168597585,0,0,22.0,1562.666667,82.0,2111.714286,118.0,1434.642857,260,1283.133333,...,14.0,2.0,0.0,0.0,0.0,0.0,1.0,3.0,14.0,14.0
168863530,1,0,0.0,0.000000,0.0,0.000000,0.0,0.000000,1,6.200000,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,90.0


### Always Check Data Integrity ! ! 

In [100]:

print("churn split in final data")
df_all['churn'].value_counts()

churn split in final data


1    11554
0    10068
Name: churn, dtype: int64

In [101]:
df_all.to_csv('../Data/sample_5pct_2.csv', header=True)

# END

In [71]:
df_win = search[(pd.to_datetime('2017-04-29') - search['search_date']).dt.days <= 1]

In [69]:
df_win.tail()

,uid,search_date_str,search_query,search_date
41316,168632256.0,2017-05-12 23:34:36,%e7%9a%87%e5%86%a0,2017-05-12 23:34:36
41317,168632256.0,2017-05-12 23:36:26,%e4%bd%a0%e7%9a%84%e7%b4%a0%e9%a2%9c,2017-05-12 23:36:26
41318,168632256.0,2017-05-12 23:37:08,%e7%9f%ad%e5%81%87%e6%9c%9f,2017-05-12 23:37:08
41319,168632256.0,2017-05-12 23:37:29,%e8%88%8d%e4%b8%8d%e5%be%97%e7%9a%84%e8%83%8c%...,2017-05-12 23:37:29
41320,168632256.0,2017-05-12 23:44:03,%e6%92%95%e5%bf%83%e6%ac%b2,2017-05-12 23:44:03


In [30]:
df_win.groupby('uid')['play_time'].agg(['count', 'sum', 'mean'])

,count,sum,mean
uid,,,
17745172.0,38,4175.0,109.868421
17972352.0,2,367.0,183.500000
19201816.0,32,6256.0,195.500000
47544816.0,5,398.0,79.600000
51908840.0,1,39907.0,39907.000000
96451488.0,58,8963.0,154.534483
132343856.0,101,30093.0,297.950495
148445200.0,37,8821.0,238.405405
150530384.0,25,4201.0,168.040000


In [ ]:
search_1 = compute_freq(search, 'search_date', '2017-04-29', 1)